# Ajuste FLAN-T5 com Aprendizado por Reforço (PPO) e PEFT para gerar resumos menos tóxicos

Neste notebook, você ajustará um modelo FLAN-T5 para gerar conteúdo menos tóxico com o modelo de recompensa por discurso de ódio da Meta AI. O modelo de recompensa é um classificador binário que prevê "não ódio" ou "ódio" para um determinado texto. Você usará a Otimização de Política Proximal (PPO) para ajustar e reduzir a toxicidade do modelo.

# Índice

- [1 - Configurar Kernel e Dependências Necessárias](#1)
- [2 - Carregar modelo FLAN-T5, preparar modelo de recompensa e avaliador de toxicidade](#2)
   - [2.1 - Carregar dados e modelo FLAN-T5 ajustado com instrução de resumo](#2.1)
   - [2.2 - Preparar modelo de recompensa](#2.2)
   - [ 2.3 - Avaliar toxicidade](#2.3)
- [3 - Execute o ajuste fino para desintoxicar os resumos](#3)
   - [3.1 - Inicializar `PPOTrainer`](#3.1)
   - [3.2 - Ajuste fino do modelo](#3.2)
   - [ 3.3 - Avalie o modelo quantitativamente](#3.3)
   - [ 3.4 - Avaliar o modelo qualitativamente](#3.4)

<a nome='1'></a>
## 1 - Configurar o kernel e as dependências necessárias

In [ ]:
!pip install torch torchdata

!pip install transformers datasets evaluate rouge_score loralib peft==0.3.0

!pip install git+https://github.com/lvwerra/trl.git@25fa1bd

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning library - É OQ DA ACESSO AO PPO
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

<a nome='2'></a>
## 2 - Carregar modelo FLAN-T5, preparar modelo de recompensa e avaliador de toxicidade

<a nome='2.1'></a>
### 2.1 - Carregar dados e modelo FLAN-T5 ajustado com instrução de resumo

Você continuará trabalhando com o mesmo conjunto de dados Hugging Face [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) e o modelo pré-treinado [FLAN-T5](https://huggingface.co/docs/ transformadores/model_doc/flan-t5).

In [ ]:
model_name="google/flan-t5-base"
huggingface_dataset_name = "knkarthick/dialogsum"

dataset_original = load_dataset(huggingface_dataset_name)

dataset_original

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

A próxima etapa será pré-processar o conjunto de dados. Você pegará apenas uma parte dele e depois filtrará os diálogos de uma duração específica (apenas para tornar esses exemplos longos o suficiente e, ao mesmo tempo, fáceis de ler). Em seguida, envolva cada diálogo com a instrução e tokenize os prompts. Salve os ids do token no campo `input_ids` e a versão decodificada dos prompts no campo `query`.

Você poderia fazer tudo isso passo a passo na célula abaixo, mas é um bom hábito organizar tudo isso em uma função `build_dataset`:

In [ ]:
def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model_name (str): Tokenizer model name.
    - dataset_name (str): Name of the dataset to load.
    - input_min_text_length (int): Minimum length of the dialogues.
    - input_max_text_length (int): Maximum length of the dialogues.

    Returns:
    - dataset_splits (datasets.dataset_dict.DatasetDict): Preprocessed dataset containing train and test parts.
    """

    # load dataset (only "train" part will be enough for this lab).
    dataset = load_dataset(dataset_name, split="train")

    # Filter the dialogues of length between input_min_text_length and input_max_text_length characters.
    dataset = dataset.filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)

    # Prepare tokenizer. Setting device_map="auto" allows to switch between GPU and CPU automatically.
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    def tokenize(sample):

        # Wrap each dialogue with the instruction.
        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)

        # This must be called "query", which is a requirement of our PPO library.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # Tokenize each dialogue.
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    # Split the dataset into train and test parts.
    dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)

    return dataset_splits

dataset = build_dataset(model_name=model_name,
                        dataset_name=huggingface_dataset_name,
                        input_min_text_length=200,
                        input_max_text_length=1000)

print(dataset)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/10022 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 8017
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 2005
    })
})


No laboratório anterior, você ajustou o modelo PEFT com instruções de resumo. O treinamento no notebook foi feito em um subconjunto de dados. Em seguida, você baixou o ponto de verificação do modelo PEFT totalmente treinado do S3.

Vamos carregar o mesmo ponto de verificação do modelo aqui:

Prepare uma função para extrair o número de parâmetros do modelo (é a mesma do laboratório anterior):

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

Adicione o adaptador ao modelo FLAN-T5 original. No laboratório anterior, você adicionou o adaptador totalmente treinado apenas para inferências, portanto não houve necessidade de passar configurações LoRA para fazer isso. Agora você precisa passá-los para o modelo PEFT construído, colocando também `is_trainable=True`.

In [ ]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              torch_dtype=torch.bfloat16)

peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'

peft_model = PeftModel.from_pretrained(model,
                                       peft_dialogue_summary_checkpoint,
                                       lora_config=lora_config,
                                       torch_dtype=torch.bfloat16,
                                       device_map="auto",
                                       is_trainable=True)

print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(peft_model)}\n')


adapter_config.json:   0%|          | 0.00/334 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

PEFT model parameters to be updated:

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%



Neste laboratório, você está se preparando para ajustar o LLM usando Reinforcement Learning (RL). RL será brevemente discutido na próxima seção deste laboratório, mas neste estágio, você só precisa preparar o modelo de Otimização de Política Proximal (PPO), passando o modelo PEFT ajustado por instrução para ele. O PPO será usado para otimizar a política de RL em relação ao modelo de recompensa.

In [ ]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

print(f'PPO model parameters to be updated (ValueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

PPO model parameters to be updated (ValueHead + 769 params):

trainable model parameters: 3539713
all model parameters: 251117569
percentage of trainable model parameters: 1.41%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


Durante o PPO, apenas alguns parâmetros serão atualizados. Especificamente, os parâmetros do `ValueHead`. Mais informações sobre esta classe de modelos podem ser encontradas na [documentação](https://huggingface.co/docs/trl/main/en/models#trl.create_reference_model). O número de parâmetros treináveis pode ser calculado como $(n+1)*m$, onde $n$ é o número de unidades de entrada (aqui $n=768$) e $m$ é o número de unidades de saída (você tem $m=1$). O termo $+1$ na equação leva em consideração o termo de polarização.

Agora crie uma cópia congelada do PPO que não será ajustada - um modelo de referência. O modelo de referência representará o LLM antes da desintoxicação. Nenhum dos parâmetros do modelo de referência será atualizado durante o treinamento do PPO. Isso é de propósito.

In [ ]:
ref_model = create_reference_model(ppo_model)

print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 251117569
percentage of trainable model parameters: 0.00%



<a nome='2.2'></a>
### 2.2 - Preparar modelo de recompensa

**Aprendizado por Reforço (RL)** é um tipo de aprendizado de máquina em que os agentes realizam ações em um ambiente que visa maximizar suas recompensas cumulativas. O comportamento do agente é definido pela **política**. E o objetivo da aprendizagem por reforço é que o agente aprenda uma política ótima, ou quase ótima, que maximize a **função de recompensa**.

Na [seção anterior](#2.1) a política original é baseada no modelo PEFT instruído - este é o LLM antes da desintoxicação. Então você poderia pedir aos rotuladores humanos que fornecessem feedback sobre a toxicidade dos resultados. No entanto, pode ser caro usá-los em todo o processo de ajuste fino. Uma forma prática de evitar isso é utilizar um modelo de recompensa que incentive o agente a desintoxicar os resumos dos diálogos. A abordagem intuitiva seria fazer alguma forma de análise de sentimento em duas classes (`nothate` e `hate`) e dar uma recompensa maior se houver maior chance de obter a classe `nothate` como saída.

Por exemplo, podemos mencionar que ter rotuladores humanos para todo o processo de ajuste fino pode ser caro. Uma maneira prática de evitar isso é usar um modelo de recompensa.

usar feedback gerado por um modelo

Você usará o [modelo de discurso de ódio baseado em RoBERTa da Meta AI](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target) para o modelo de recompensa. Este modelo produzirá **logits** e então preverá probabilidades em duas classes: `nothate` e `hate`. Os logits da saída `nothate` serão considerados uma recompensa positiva. Em seguida, o modelo será ajustado com PPO usando esses valores de recompensa.

Crie a instância da classe de modelo necessária para o modelo RoBERTa. Você também precisa carregar um tokenizer para testar o modelo. Observe que o rótulo do modelo `0` corresponderá à classe `nothate` e o rótulo `1` à classe `hate`.

In [ ]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map="auto")
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto")
print(toxicity_model.config.id2label)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

{0: 'nothate', 1: 'hate'}


Pegue algum texto não tóxico, tokenize-o e passe-o para o modelo. Imprima os logits de saída, as probabilidades e a recompensa correspondente que será usada para o ajuste fino.

In [ ]:
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."

toxicity_input_ids = toxicity_tokenizer(non_toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(input_ids=toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# get the logits for "not hate" - this is the reward!
not_hate_index = 0
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (high): {nothate_reward}')

logits [not hate, hate]: [3.114103078842163, -2.489619731903076]
probabilities [not hate, hate]: [0.9963293671607971, 0.0036706007085740566]
reward (high): [3.114103078842163]


Let's show a toxic comment.  This will have a low reward because it is more toxic.

In [ ]:
toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."

toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# Get the logits for "not hate" - this is the reward!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (low): {nothate_reward}')

logits [not hate, hate]: [-0.6921194195747375, 0.3722732961177826]
probabilities [not hate, hate]: [0.2564708888530731, 0.7435290813446045]
reward (low): [-0.6921194195747375]


Configure o pipeline de inferência Hugging Face para simplificar o código do modelo de recompensa de toxicidade:

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"

sentiment_pipe = pipeline("sentiment-analysis",
                          model=toxicity_model_name,
                          device=device)
reward_logits_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # Set to "none" to retrieve raw logits.
    "batch_size": 16
}

reward_probabilities_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "softmax", # Set to "softmax" to apply softmax and retrieve probabilities.
    "batch_size": 16
}

print("Reward model output:")
print("For non-toxic text")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))
print("For toxic text")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

Reward model output:
For non-toxic text
[{'label': 'nothate', 'score': 3.114103078842163}, {'label': 'hate', 'score': -2.489619731903076}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.0036706007085740566}]
For toxic text
[{'label': 'hate', 'score': 0.3722732961177826}, {'label': 'nothate', 'score': -0.6921194195747375}]
[{'label': 'hate', 'score': 0.7435290813446045}, {'label': 'nothate', 'score': 0.2564708888530731}]


As saídas são os logits para as classes `nothate` (positiva) e `hate` (negativa). Mas o PPO usará logits apenas da classe `nothate` como sinal de recompensa positivo usado para ajudar a desintoxicar os resultados do LLM.

In [ ]:
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))

[{'label': 'nothate', 'score': 3.114103078842163}, {'label': 'hate', 'score': -2.489619731903076}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.0036706007085740566}]


In [ ]:
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

[{'label': 'hate', 'score': 0.3722732961177826}, {'label': 'nothate', 'score': -0.6921194195747375}]
[{'label': 'hate', 'score': 0.7435290813446045}, {'label': 'nothate', 'score': 0.2564708888530731}]


<a nome='2.3'></a>
### 2.3 - Avaliar Toxicidade

Para avaliar o modelo antes e depois do ajuste fino/desintoxicação, você precisa configurar a [métrica de avaliação de toxicidade](https://huggingface.co/spaces/evaluate-measurement/toxicity). A **pontuação de toxicidade** é um valor decimal entre 0 e 1, onde 1 é a toxicidade mais alta.

In [ ]:
toxicity_evaluator = evaluate.load("toxicity",
                                    toxicity_model_name,
                                    module_type="measurement",
                                    toxic_label="hate")

Tente calcular a toxicidade para as mesmas frases da seção [2.2](#2.2). Não é nenhuma surpresa que as pontuações de toxicidade sejam as probabilidades da classe “ódio” retornadas diretamente do modelo de recompensa.

In [ ]:
toxicity_score = toxicity_evaluator.compute(predictions=[
    non_toxic_text
])

print("Toxicity score for non-toxic text:")
print(toxicity_score["toxicity"])

toxicity_score = toxicity_evaluator.compute(predictions=[
    toxic_text
])

print("\nToxicity score for toxic text:")
print(toxicity_score["toxicity"])

Toxicity score for non-toxic text:
[0.0036706007085740566]

Toxicity score for toxic text:
[0.7435290813446045]


Este avaliador pode ser usado para calcular a toxicidade dos diálogos preparados na seção [2.1](#2.1). Você precisará passar no conjunto de dados de teste (`dataset["test"]`), no mesmo tokenizer usado naquela seção, no modelo PEFT congelado preparado na seção [2.2](#2.2) e no avaliador de toxicidade. É conveniente agrupar as etapas necessárias na função `evaluate_toxicity`.

In [ ]:
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model (trl model): Model to be evaluated.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
    - tokenizer (transformers tokenizer): Tokenizer to be used.
    - dataset (dataset): Input dataset for the evaluation.
    - num_samples (int): Maximum number of samples for the evaluation.

    Returns:
    tuple: A tuple containing two numpy.float64 values:
    - mean (numpy.float64): Mean of the samples toxicity.
    - std (numpy.float64): Standard deviation of the samples toxicity.
    """

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["query"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             top_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

        toxicities.extend(toxicity_score["toxicity"])

    # Compute mean & std using np.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

E agora realize o cálculo da toxicidade do modelo antes do ajuste fino/desintoxicação:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=toxicity_evaluator,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=10)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

11it [05:05, 27.78s/it]

toxicity [mean, std] before detox: [0.02307206247827377, 0.02775749222980828]


<a nome='3'></a>
## 3 - Execute o ajuste fino para desintoxicar os resumos
Otimize uma política de RL em relação ao modelo de recompensa usando Proximal Policy Optimization (PPO).

<a nome='3.1'></a>
### 3.1 - Inicializar `PPOTrainer`

Para a inicialização do `PPOTrainer`, você precisará de um agrupador. Aqui será uma função que transforma os dicionários de uma forma particular. Você pode defini-lo e testá-lo:

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


Configure os parâmetros de configuração. Carregue o `ppo_model` e o tokenizer. Você também carregará uma versão congelada do modelo `ref_model`. O primeiro modelo é otimizado enquanto o segundo modelo serve como referência para calcular a divergência KL a partir do ponto inicial. Isso funciona como um sinal de recompensa adicional no treinamento PPO para garantir que o modelo otimizado não se desvie muito do LLM original.

In [ ]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

<a name='3.2'></a>
### 3.2 - Fine-Tune the Model

O ciclo de ajuste fino consiste nas seguintes etapas principais:
1. Obtenha as respostas da consulta da política LLM (modelo PEFT).
2. Obtenha sentimentos para consultas/respostas do modelo RoBERTa de discurso de ódio.
3. Otimize a política com PPO usando o trio (consulta, resposta, recompensa).

A operação estará em execução se você vir as seguintes métricas aparecendo:
* `objetivo/kl`: minimizar a divergência kl,
* `ppo/returns/mean`: maximiza os retornos médios,
* `ppo/policy/advantages_mean`: maximiza vantagens.

In [ ]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # You want the raw logits without softmax.
    "batch_size": 16
}

max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Break when you reach max_steps.
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

    # You use the `nothate` item because this is the score for the positive `nothate` class.
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

1it [1:07:43, 4063.84s/it]

objective/kl: 30.36363983154297
ppo/returns/mean: -0.4685230255126953
ppo/policy/advantages_mean: -1.8282419134152406e-08
---------------------------------------------------------------------------------------------------


2it [2:10:38, 3893.59s/it]

objective/kl: 36.099693298339844
ppo/returns/mean: -0.7869019508361816
ppo/policy/advantages_mean: 1.8399408219238467e-08
---------------------------------------------------------------------------------------------------


<a nome='3.3'></a>
### 3.3 - Avalie o modelo quantitativamente

Carregue o modelo PPO/PEFT novamente do disco e use a divisão do conjunto de dados de teste para avaliar a pontuação de toxicidade do modelo ajustado por RL.

In [ ]:
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=toxicity_evaluator,
                                                                        tokenizer=tokenizer,
                                                                        dataset=dataset["test"],
                                                                        num_samples=10)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')

11it [00:20,  1.90s/it]

toxicity [mean, std] after detox: [0.041582453159869394, 0.05383645491759948]


E compare as pontuações de toxicidade do modelo de referência (antes da desintoxicação) e do modelo ajustado (após a desintoxicação).

In [ ]:
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of toxicity score after detoxification:
mean: -5.76%
std: -12.18%


<a nome='3.4'></a>
### 3.4 - Avalie o modelo qualitativamente

Vamos inspecionar alguns exemplos do conjunto de dados de teste. Você pode comparar o `ref_model` original com o `ppo_model` ajustado/desintoxicado usando o avaliador de toxicidade.

In [ ]:
batch_size = 20
compare_results = {}

df_batch = dataset["test"][0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

100%|██████████| 20/20 [01:18<00:00,  3.91s/it]


Armazene e revise os resultados em um DataFrame

In [ ]:
pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Summarize the following conversation. #Person1#: Today more and more families have personal computers. People have wider range of choice to communicate with the outside world. #Person2#: Right. With the establishment of Internet and a lot of web companies, people are getting more and more dependent on the web. #Person1#: One of the common uses of PC is that people can buy goods through it without going out to the physical stores. #Person2#: Can you tell me how it is done? #Person1#: If a cus...","<pad> #Person1# tells #Person2# how people are getting more and more dependent on the web. They tell them, “Standard computers are the green hand of the Internet, drives are great, and most products can be delivered directly.</s>",<pad> #Person2# explains the advantages of computers and current use of the Internet. It shows common uses for personal computers that he can buy goods through the internet where he can place an order online. They are very happy.</s>,2.351077,2.948620,0.597543
1,"Summarize the following conversation. #Person1#: So how did you like the restaurant? #Person2#: Actually, it could have been better. #Person1#: What didn't you like about it? #Person2#: It is a new restaurant. I don't think they have their act together yet. #Person1#: What did you think about the food? #Person2#: I felt that the food was pretty mediocre. #Person1#: The service wasn't that great, either. #Person2#: I agree. The service was not good. #Person1#: Do you think that you want to tr...",<pad> #Person2# agrees with #Person1# about the restaurant and the food. #Person1# reckons #Person2# will not return but #Person2# isn't even considering to try again.</s>,<pad> #Person1# shows #Person2# how the restaurant was turned down by the new owners. One of the other people says it's mediocre and they both say it's time to switch it.</s>,1.883278,2.461895,0.578617
2,"Summarize the following conversation. #Person1#: Hello. I want to reconfirm our flight to London. #Person2#: Yes, sir. Did you call the airline? #Person1#: Yes, I did. But I couldn't communicate with them in English. They speak only Spanish. So I need your help. #Person2#: Certainly, sir. What is the flight number and when are you leaving? #Person1#: We are taking IB 385 to London tomorrow at 1 p. m. #Person2#: Oh, I see, sir. We have the airline office inside the hotel. They have an English...",<pad> #Person2# gives #Person1# a number of flights from #Person1# to London by calling 35 and intervenes. #Person2# disapproves and offers to help.</s>,<pad> The airline will reconfirm their flight tomorrow and they dial 35. #Person1# asks our airline office about flight number and flight time.</s>,1.587816,1.953090,0.365274
3,"Summarize the following conversation. #Person1#: Let's take a coffee break, shall we? #Person2#: I wish I could, but I can't. #Person1#: What keeps you so busy? You've been sitting there for hours. You've got to walk around. You just can't stay on the computer forever. #Person2#: Well, I am up to my neck in work. I've got to finish this report. Sarah needs it by noon. I don't want to be scolded if I can't finish my work by the deadline. #Person1#: I understand that, but you'd feel better if ...",<pad> #Person1# offers #Person2# a coffee break while #Person2# works because #Person2# is up to their neck in work.</s>,<pad> #Person1# and #Person2# are too busy to take a coffee break. They ask for a coffee break but #Person2# doesn't.</s>,1.611620,1.907171,0.295550
4,"Summarize the following conversation. #Person1#: Amanda, how do you like this peaked cap? #Person2#: Didn't you say you want to buy a top hat? #Person1#: But I think this one fits me Well. Why don't you try on the sombrero in black? #Person2#: I don't like caps at all. Summary: </s>","<pad> Amanda chooses a peaked cap, but doesn't like the sombrero in black. #Person2# might consider a pig for Amanda.</s>","<pad> Amanda likes her trendy

Olhando para a média/mediana de recompensa das sequências geradas você pode observar uma diferença significativa!